In [1]:

import cv2
from PIL import Image, ImageDraw, ImageFont, ImageTransform
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from glob import glob
import sys

import os
from configparser import ConfigParser
path_config_file = os.getcwd().split('docrv2_sroie')[0] + 'docrv2_sroie/' + 'config.ini'
common_variable = ConfigParser()
common_variable.read(path_config_file)
print(common_variable.sections())
print(common_variable['database']['module_path'])
sys.path.append(common_variable['database']['module_path'])
from db_util import *
from db import *
from random import choice
from cyclegan import CycleGAN
import tensorflow as tf

['database', 'mlflow', 'font']
/home/dk/docrv2_sroie/modules/database/
['database', 'mlflow', 'font']
/home/dk/docrv2_sroie/modules/database/


Using TensorFlow backend.


'style_real_00020_A'

In [8]:
# set simulator parameter
modeldir = '/home/dk/docrv2_sroie/modules/simulator/cyclegan/models/'
modelpath = glob(modeldir+'*_BA.h5')
modelpath.append('None')
def load_model(path):
    if path is not 'None':
        gan = CycleGAN(dataset_name='inference')
        gan.g_AB.load_weights(path)
        return gan
    else:
        return 'None'

def break_model(gan):
    if gan is not 'None':
        tf.keras.backend.clear_session()
        del gan

def inference_batch(gan, batch):
    if gan is not 'None':
        batch = np.array(batch)/127.5-1.
        batch = gan.g_AB.predict(batch)
        batch = 0.5*batch+0.5
        batch = np.uint8(batch*255)
    return batch

In [56]:
# apply cycleGAN later here
path = '/home/dk/docrv2_sroie/DB/'
name = 'denoise_train_100000_nostyle'
db_data = open_env(path+name)
img_raw = read_bulk_data_from_db(db_data,'img')
imgs = []
for idx in range(0,len(img_raw),2):
    imgs.append(img_raw[idx])


In [4]:
# some generator
def make_batch_image(imgs, batch_size):
    numImages = len(imgs)
    numbatch = int(np.ceil(numImages/batch_size))
    imgsize = imgs[0].size

    batchImages = np.zeros((batch_size,imgsize[1],imgsize[0],3), dtype=np.uint8)
    bidx = 0
    for img in imgs:
        if bidx == batch_size:
            batchImages = np.zeros((batch_size,imgsize[1],imgsize[0],3), dtype=np.uint8)
            bidx = 0
        batchImages[bidx,:] = np.array(img, dtype=np.uint8)
        bidx = bidx + 1
        if bidx == batch_size:
            yield batchImages


In [57]:
batch_size = 100
numImages = len(imgs)
numbatch = int(np.ceil(numImages/batch_size))
batchgen = make_batch_image(imgs, batch_size)
imglist = []
for cidx in tqdm(range(numbatch)):
    batchim = next(batchgen)
    gan = load_model(choice(modelpath))
    
    # apply cyclegan
    batchim_inf = inference_batch(gan, batchim)
        
    # make image list
    for bidx in range(batch_size):
        img = Image.fromarray(batchim_inf[bidx,])
        imglist.append(img)

    # remove model
    break_model(gan)


  0%|          | 0/701 [00:00<?, ?it/s]


  0%|          | 1/701 [00:02<24:18,  2.08s/it]


  0%|          | 2/701 [00:03<23:29,  2.02s/it]


 23%|██▎       | 159/701 [05:09<17:17,  1.91s/it]


 45%|████▍     | 314/701 [10:12<12:08,  1.88s/it]


 67%|██████▋   | 469/701 [15:15<07:25,  1.92s/it]


 89%|████████▉ | 623/701 [20:22<02:32,  1.96s/it]


100%|█████████▉| 700/701 [22:54<00:01,  1.87s/it]

StopIteration: 

In [60]:
len(img_raw)

140002

In [62]:
#save image to db
ind = 0
for idx in range(0,len(img_raw)-2,2):
    img_raw[idx] = imglist[ind]
    ind = ind + 1
    
#name = name+'_styled'
name = 'denoise_train_100000_styled'
cache = {}
db_data = create_data_db(path=path, name=name, description='datadb_denoising',data_class='denoising')
db_data_db = db_data.open_db(b'db_data')
for idx in tqdm(range(len(img_raw))):
    img = img_raw[idx]
    cache = update_cache(cache, index=idx, label='denoising', ref=name, image=img)
    if idx % 5000 == 0:
        write_cache_to_db(db_data, db_data_db, cache)
        cache = {}
        
write_cache_to_db(db_data, db_data_db, cache)
cache = {}



  0%|          | 0/140002 [00:00<?, ?it/s]

  0%|          | 185/140002 [00:00<01:15, 1848.27it/s]

  0%|          | 358/140002 [00:00<01:17, 1810.26it/s]

  0%|          | 526/140002 [00:00<01:18, 1768.66it/s]

  0%|          | 687/140002 [00:00<01:21, 1716.83it/s]

  1%|          | 849/140002 [00:00<01:22, 1682.79it/s]

  1%|          | 1018/140002 [00:00<01:22, 1683.06it/s]

  1%|          | 1188/140002 [00:00<01:22, 1687.20it/s]

  1%|          | 1367/140002 [00:00<01:20, 1715.87it/s]

  1%|          | 1544/140002 [00:00<01:20, 1730.10it/s]

  1%|          | 1720/140002 [00:01<01:19, 1736.50it/s]

  1%|▏         | 1889/140002 [00:01<01:23, 1663.77it/s]

  1%|▏         | 2053/140002 [00:01<01:27, 1582.46it/s]

  2%|▏         | 2227/140002 [00:01<01:24, 1624.83it/s]

  2%|▏         | 2409/140002 [00:01<01:22, 1677.27it/s]

  2%|▏         | 2577/140002 [00:01<01:25, 1599.26it/s]

  2%|▏         | 2738/140002 [00:01<01:29, 1532.71it/s]

  2%|▏         | 2906/140002 [00:01<01:27, 1565

 16%|█▌        | 22245/140002 [00:13<01:10, 1681.74it/s]

 16%|█▌        | 22421/140002 [00:13<01:09, 1702.27it/s]

 16%|█▌        | 22601/140002 [00:13<01:07, 1728.89it/s]

 16%|█▋        | 22775/140002 [00:14<01:08, 1701.56it/s]

 16%|█▋        | 22955/140002 [00:14<01:07, 1727.83it/s]

 17%|█▋        | 23141/140002 [00:14<01:06, 1763.68it/s]

 17%|█▋        | 23322/140002 [00:14<01:05, 1776.57it/s]

 17%|█▋        | 23505/140002 [00:14<01:05, 1789.25it/s]

 17%|█▋        | 23694/140002 [00:14<01:04, 1816.75it/s]

 17%|█▋        | 23888/140002 [00:14<01:02, 1851.87it/s]

 17%|█▋        | 24074/140002 [00:14<01:02, 1845.15it/s]

 17%|█▋        | 24259/140002 [00:14<01:03, 1821.90it/s]

 17%|█▋        | 24443/140002 [00:14<01:03, 1825.80it/s]

 18%|█▊        | 24626/140002 [00:15<01:03, 1825.16it/s]

 18%|█▊        | 24812/140002 [00:15<01:02, 1834.46it/s]

 18%|█▊        | 24996/140002 [00:15<01:04, 1782.31it/s]

 18%|█▊        | 25175/140002 [00:15<01:22, 1396.48it/s]

 18%|█▊       

 32%|███▏      | 45169/140002 [00:27<01:10, 1343.94it/s]

 32%|███▏      | 45321/140002 [00:27<01:08, 1374.38it/s]

 32%|███▏      | 45492/140002 [00:27<01:04, 1460.09it/s]

 33%|███▎      | 45673/140002 [00:27<01:00, 1549.53it/s]

 33%|███▎      | 45837/140002 [00:27<01:02, 1504.56it/s]

 33%|███▎      | 46020/140002 [00:27<00:59, 1589.34it/s]

 33%|███▎      | 46185/140002 [00:28<01:00, 1547.18it/s]

 33%|███▎      | 46344/140002 [00:28<01:02, 1505.69it/s]

 33%|███▎      | 46504/140002 [00:28<01:01, 1532.00it/s]

 33%|███▎      | 46681/140002 [00:28<00:58, 1595.98it/s]

 33%|███▎      | 46843/140002 [00:28<00:59, 1565.85it/s]

 34%|███▎      | 47002/140002 [00:28<01:00, 1527.77it/s]

 34%|███▎      | 47190/140002 [00:28<00:57, 1617.89it/s]

 34%|███▍      | 47362/140002 [00:28<00:56, 1645.45it/s]

 34%|███▍      | 47531/140002 [00:28<00:55, 1657.35it/s]

 34%|███▍      | 47704/140002 [00:28<00:55, 1677.04it/s]

 34%|███▍      | 47875/140002 [00:29<00:54, 1685.95it/s]

 34%|███▍     

 48%|████▊     | 67486/140002 [00:41<00:41, 1767.71it/s]

 48%|████▊     | 67668/140002 [00:41<00:40, 1780.85it/s]

 48%|████▊     | 67854/140002 [00:41<00:40, 1801.98it/s]

 49%|████▊     | 68035/140002 [00:41<00:40, 1784.53it/s]

 49%|████▊     | 68214/140002 [00:41<00:41, 1714.99it/s]

 49%|████▉     | 68387/140002 [00:41<00:42, 1673.42it/s]

 49%|████▉     | 68561/140002 [00:41<00:42, 1692.02it/s]

 49%|████▉     | 68731/140002 [00:41<00:43, 1623.51it/s]

 49%|████▉     | 68907/140002 [00:41<00:42, 1661.55it/s]

 49%|████▉     | 69083/140002 [00:42<00:42, 1688.14it/s]

 49%|████▉     | 69259/140002 [00:42<00:41, 1707.81it/s]

 50%|████▉     | 69438/140002 [00:42<00:40, 1729.99it/s]

 50%|████▉     | 69628/140002 [00:42<00:39, 1777.64it/s]

 50%|████▉     | 69818/140002 [00:42<00:38, 1810.97it/s]

 50%|████▉     | 70000/140002 [00:42<00:38, 1810.20it/s]

 50%|█████     | 70182/140002 [00:42<00:49, 1407.54it/s]

 50%|█████     | 70368/140002 [00:42<00:45, 1516.83it/s]

 50%|█████    

 64%|██████▍   | 90027/140002 [00:54<00:41, 1212.96it/s]

 64%|██████▍   | 90195/140002 [00:54<00:37, 1322.99it/s]

 65%|██████▍   | 90338/140002 [00:55<00:37, 1338.32it/s]

 65%|██████▍   | 90520/140002 [00:55<00:34, 1452.70it/s]

 65%|██████▍   | 90697/140002 [00:55<00:32, 1534.70it/s]

 65%|██████▍   | 90869/140002 [00:55<00:30, 1585.43it/s]

 65%|██████▌   | 91051/140002 [00:55<00:29, 1647.94it/s]

 65%|██████▌   | 91234/140002 [00:55<00:28, 1697.72it/s]

 65%|██████▌   | 91412/140002 [00:55<00:28, 1720.22it/s]

 65%|██████▌   | 91602/140002 [00:55<00:27, 1769.07it/s]

 66%|██████▌   | 91781/140002 [00:55<00:27, 1749.74it/s]

 66%|██████▌   | 91964/140002 [00:55<00:27, 1772.93it/s]

 66%|██████▌   | 92143/140002 [00:56<00:27, 1764.74it/s]

 66%|██████▌   | 92325/140002 [00:56<00:26, 1779.00it/s]

 66%|██████▌   | 92506/140002 [00:56<00:26, 1787.93it/s]

 66%|██████▌   | 92686/140002 [00:56<00:26, 1781.09it/s]

 66%|██████▋   | 92870/140002 [00:56<00:26, 1796.76it/s]

 66%|██████▋  

 80%|████████  | 112422/140002 [01:08<00:16, 1713.95it/s]

 80%|████████  | 112607/140002 [01:08<00:15, 1750.71it/s]

 81%|████████  | 112784/140002 [01:08<00:15, 1711.80it/s]

 81%|████████  | 112961/140002 [01:08<00:15, 1728.39it/s]

 81%|████████  | 113135/140002 [01:08<00:16, 1651.03it/s]

 81%|████████  | 113315/140002 [01:08<00:15, 1691.79it/s]

 81%|████████  | 113490/140002 [01:09<00:15, 1706.28it/s]

 81%|████████  | 113663/140002 [01:09<00:15, 1710.07it/s]

 81%|████████▏ | 113836/140002 [01:09<00:15, 1714.75it/s]

 81%|████████▏ | 114008/140002 [01:09<00:15, 1704.26it/s]

 82%|████████▏ | 114179/140002 [01:09<00:15, 1698.96it/s]

 82%|████████▏ | 114350/140002 [01:09<00:15, 1636.19it/s]

 82%|████████▏ | 114515/140002 [01:09<00:16, 1584.18it/s]

 82%|████████▏ | 114693/140002 [01:09<00:15, 1634.33it/s]

 82%|████████▏ | 114858/140002 [01:09<00:15, 1592.95it/s]

 82%|████████▏ | 115019/140002 [01:10<00:19, 1261.56it/s]

 82%|████████▏ | 115188/140002 [01:10<00:18, 1364.75it/s

 96%|█████████▋| 134896/140002 [01:22<00:03, 1551.53it/s]

 96%|█████████▋| 135056/140002 [01:22<00:05, 947.02it/s] 

 97%|█████████▋| 135237/140002 [01:22<00:04, 1104.97it/s]

 97%|█████████▋| 135410/140002 [01:22<00:03, 1238.42it/s]

 97%|█████████▋| 135598/140002 [01:22<00:03, 1379.33it/s]

 97%|█████████▋| 135786/140002 [01:22<00:02, 1498.75it/s]

 97%|█████████▋| 135969/140002 [01:22<00:02, 1583.56it/s]

 97%|█████████▋| 136148/140002 [01:22<00:02, 1638.15it/s]

 97%|█████████▋| 136326/140002 [01:23<00:02, 1677.21it/s]

 98%|█████████▊| 136502/140002 [01:23<00:02, 1525.14it/s]

 98%|█████████▊| 136663/140002 [01:23<00:02, 1502.80it/s]

 98%|█████████▊| 136837/140002 [01:23<00:02, 1566.65it/s]

 98%|█████████▊| 137019/140002 [01:23<00:01, 1632.22it/s]

 98%|█████████▊| 137187/140002 [01:23<00:01, 1465.92it/s]

 98%|█████████▊| 137340/140002 [01:23<00:01, 1466.83it/s]

 98%|█████████▊| 137523/140002 [01:23<00:01, 1558.20it/s]

 98%|█████████▊| 137708/140002 [01:23<00:01, 1635.06it/s

In [63]:
print_env(db_data)
refresh_main_db()

+-------------+------------------------------------------------------+
| Key         | Value                                                |
+-------------+------------------------------------------------------+
| class       | denoising                                            |
| created     | 2020-08-19 10:31:40                                  |
| db_data     | (database)                                           |
| description | datadb_denoising                                     |
| name        | denoise_train_100000_styled                          |
| note        |                                                      |
| reference   | /home/dk/docrv2_sroie/DB/denoise_train_100000_styled |
| sep         | 	                                                    |
| updated     | 2020-08-19 10:31:40                                  |
+-------------+------------------------------------------------------+
+------------------------------+---------------------------------------------

['style_real_00293',
 'style_real_00092',
 'style_real_00217',
 'style_real_00347',
 'style_real_00250',
 'style_real_00342',
 'style_real_00190',
 'style_real_00245',
 'style_real_00060',
 'style_real_00358',
 'denoise_test_100_styled',
 'style_real_00077',
 'style_real_00009',
 'style_real_00201',
 'style_real_00355',
 'style_real_00325',
 'style_real_00279',
 'style_real_00089',
 'style_real_00268',
 'style_real_00085',
 'style_real_00082',
 'style_real_00316',
 'style_real_00163',
 'style_real_00137',
 'style_real_00155',
 'denoise_train_100000_nostyle',
 'style_real_00308',
 'style_real_00213',
 'style_real_00255',
 'style_real_00298',
 'style_real_00220',
 'style_real_00045',
 'style_real_00047',
 'style_real_00334',
 'style_real_00132',
 'style_real_00254',
 'style_real_00134',
 'style_real_00295',
 'style_real_00108',
 'style_real_00246',
 'style_real_00062',
 'style_real_00196',
 'style_real_00176',
 'style_real_00167',
 'style_real_00257',
 'style_real_00117',
 'style_real_00